In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [6]:
# Read LA_u subsidized
la = pd.read_csv('los_angeles_u_v2'+'.csv', sep='|')

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,14,16,17,19,22,30,40,43,45,48,51,53,56,57,60,63,73,74,79,92,99,100,103,120,121,123,124,131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Filter for Santa Monica JCE only
sm_jc = la[la.PropertyCity == 'SANTA MONICA']
print('Current number:', len(sm_jc))
la.PropertyCity.value_counts()

Current number: 22021


LOS ANGELES              388003
LONG BEACH                99415
SANTA CLARITA             57713
LANCASTER                 50937
PALMDALE                  48377
WHITTIER                  44664
TORRANCE                  43778
PASADENA                  40468
GLENDALE                  39953
POMONA                    30458
BURBANK                   26831
LA PUENTE                 26491
WEST COVINA               26000
NORTH HOLLYWOOD           24793
COMPTON                   24499
VAN NUYS                  24189
LAKEWOOD                  23668
REDONDO BEACH             23221
CARSON                    22584
WOODLAND HILLS            22273
NORWALK                   22175
SANTA MONICA              22021
DOWNEY                    21975
INGLEWOOD                 21649
SYLMAR                    20718
COVINA                    20004
SHERMAN OAKS              19257
ARCADIA                   18344
ALHAMBRA                  17776
DIAMOND BAR               17660
                          ...  
THOUSAND

In [8]:
# Filter for not subsidized
display(sm_jc.head(1))
sm_jc = sm_jc[sm_jc['Subsidized'] == 'N']

,ImportParcelID,RowID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode,merge_row,Subsidized
453952,11628009,4219EC49-D6D5-E511-80C1-3863BB43AC67,6037,CA,LOS ANGELES,2015-07-01,72015,23,BKF,3009-009-003,NaN,1,NaN,NaN,NaN,,NaN,1158,NaN,NaN,26TH,ST,NaN,1158 26TH ST,SANTA MONICA,CA,90403.0,NaN,NaN,NaN,PDC3*,NaN,NaN,NaN,748.57,2015.0,,NaN,NaN,6-955,51,NaN,NaN,NaN,NaN,TOWN & SUBURB OF PALMDALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGION/CLUSTER: 24/24615,NaN,NaN,NaN,NaN,NaN,NaN,TOWN AND SUBURBS OF PALMDALE LOT COM AT 1NTERSECT1ON OF W L1NE OF 6TH ST E W1TH S L1NE OF LOT 51...,NaN,0.0,0.195,8524.0,NaN,NaN,NaN,NaN,2187140790,Y,#,210,C029,Y,34.036513,-118.477356,6.0377e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,493707.0,-1.0,4219EC49-D6D5-E511-80C1-3863BB43AC67,0.0,NaN,VACANT RESIDENTIAL LAND,010V,VL101,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,C,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453952,N
557856,11759972,65F3BD4A-D6D5-E511-80C1-3863BB43AC67,6037,CA,LOS ANGELES,2015-07-01,72015,23,BKF,3214-031-016,NaN,1,NaN,NaN,NaN,,NaN,85,NaN,NaN,VICENTE,TER,NaN,85 VICENTE TER,SANTA MONICA,CA,90401.0,NaN,NaN,NaN,LCA11*,NaN,NaN,NaN,93.27,2015.0,,NaN,NaN,8-933,370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7773,NaN,NaN,NaN,REGION/CLUSTER: 01/01038,NaN,NaN,NaN,NaN,NaN,NaN,TRACT NO 7773 LOT 370,NaN,0.0,0.115,5021.0,NaN,NaN,NaN,NaN,2187273921,Y,NaN,NaN,C001,Y,34.007933,-118.490404,6.0377e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,493707.0,-1.0,65F3BD4A-D6D5-E511-80C1-3863BB43AC67,0.0,NaN,VACANT RESIDENTIAL LAN

In [9]:
# Check structure 
# (number of records, number of columns)
sm_jc.shape

(22021, 135)

In [10]:
data_dict = pd.read_excel('ZAsmt_DataDictionary_2016-01.xlsx', sheet_name='LandUse', header=5, usecols=np.arange(4))
data_dict.head(2)

,StndCode,PropertyLandUse Description,Prefix Code Classification,Classification
0,AG000,AGRICULTURAL GENERAL,AG,Agricultural
1,AG101,FARM (IRRIGATED OR DRY),AG,Agricultural


In [11]:
residential = data_dict[data_dict['Prefix Code Classification'].isin(['RI', 'RR'])]
R = residential['StndCode']

In [13]:
# Quoted comments refer to the lines in the JC and RC inventory. Specifically each coverage/exemption.
# EXEMPTIONS:

"""
-Hotels, motels, inns, tourist homes and rooming and boarding houses which are rented primarily to transient guests for a period of less than fourteen (14) days.
-Hospital, convent, monastery, extended medical care facility, asylum, non-profit home for the aged, or dormitory owned and operated by an institution of higher education.
-Government unit, agency or authority owns, operates, manages, or in which governmentally subsidized tenants reside only if applicable Federal or State law or administrative regulation specially exempt such units from municipal rent control.
"""
sm_jc = sm_jc[sm_jc['PropertyLandUseStndCode'].isin(R)]
print('Current number:', len(sm_jc))

Current number: 21751


In [16]:
"""
-Owner-occupied dwellings with no more than three (3) units.
"""

sm_jc = sm_jc[~(sm_jc['OccupancyStatusStndCode'].isin(['O', 'I']) & ((sm_jc.NoOfUnits  <= 3) | (sm_jc.PropertyLandUseStndCode.isin(['RI101', 'RI102']))))]
print('Current number:', len(sm_jc))

Current number: 8194


In [17]:
# -Rental units and dwellings constructed after the adoption of this Article; this exemption does not apply to units created as a result of conversion as opposed to new construction.
sm_jc = sm_jc[sm_jc.YearBuilt < 1980]
print('Last Current number:', len(sm_jc))

Last Current number: 6710


In [19]:
sm_rc = la[la.PropertyCity == 'SANTA MONICA']
print('Current number:', len(sm_rc))

Current number: 22021


In [20]:
# Filter for not subsidized
sm_rc = sm_rc[sm_rc['Subsidized'] == 'N']
print('Current number:', len(sm_rc))

Current number: 22021


In [22]:
# Quoted comments refer to the lines in the JC and RC inventory. Specifically each coverage/exemption.

# Filter rows to only include residential buildings
sm_rc = sm_rc[sm_rc['PropertyLandUseStndCode'].isin(R)]
print('Current number:', len(sm_rc))

Current number: 21751


In [23]:
"""
Eligible units for RC: any unit built before 1979, residential;
"""
sm_rc = sm_rc[sm_rc.YearBuilt < 1979]
print('Current number:', len(sm_rc))

Current number: 16586


In [24]:
print('Last JC Current number:', len(sm_jc))
# includes full and partially coverage
print('Last RC Current number:', len(sm_rc))

Last JC Current number: 6710
Last RC Current number: 16586
